<a href="https://colab.research.google.com/github/peterchang0414/lecun1989-flax/blob/main/lecun_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flax

     |████████████████████████████████| 184 kB 14.8 MB/s 
     |████████████████████████████████| 136 kB 54.0 MB/s 
     |████████████████████████████████| 70 kB 8.4 MB/s 


In [3]:
import jax
import jax.numpy as jnp
import numpy as np

from flax import linen as nn
from torchvision import datasets

data = datasets.MNIST('./data', train=True, download=True)

# Adapted from https://github.com/karpathy/lecun1989-repro/blob/master/prepro.py
def get_datasets():
  train_test = {}
  for split in {'train', 'test'}:
    data = datasets.MNIST('./data', train=split=='train', download=True)
    
    n = 2 if split == 'train' else 1
    key = jax.random.PRNGKey(0)
    rp = jax.random.permutation(key, len(data))[:n]

    X = jnp.full((n, 1, 16, 16), 0.0, dtype=jnp.float32)
    Y = jnp.full((n, 10), -1.0, dtype=jnp.float32)
    for i, ix in enumerate(rp):
      I, yint = data[int(ix)]
      xi = jnp.array(I, dtype=np.float32) / 127.5 - 1.0
      xi = jax.image.resize(xi, (16, 16), 'bilinear')
      X = X.at[i].set(xi)
      Y = Y.at[i, yint].set(1.0)
    train_test[split] = (X, Y)
  return train_test

In [4]:
from flax import linen as nn
from flax.training import train_state
from flax.linen.activation import tanh
import optax
from typing import Callable

class Net(nn.Module):
  bias_init: Callable = nn.initializers.zeros

  @nn.compact
  def __call__(self, x):
    # H1 layer
    # For weight initialization, Karpathy used numerator of 2.4 
    # which is very close to sqrt(6) used by he_uniform
    # By default, weight-sharing forces bias-sharing and therefore
    # we add the bias separately.
    bias1 = self.param('bias1', self.bias_init, (8, 8, 12))
    bias2 = self.param('bias2', self.bias_init, (4, 4, 12))
    bias3 = self.param('bias3', self.bias_init, (30,))
    x = nn.Conv(features=12, kernel_size=(5,5), strides=2, padding=-1.0,
                use_bias=False, kernel_init=nn.initializers.he_uniform())(x)
    x = tanh(x + bias1)
    slice1 = nn.Conv(features=8, kernel_size=(5,5), strides=2, padding=-1.0,
                     use_bias=False, kernel_init=nn.initializers.he_uniform())(x[..., 0:8])
    slice1 = nn.Conv(features=8, kernel_size=(5,5), strides=2, padding=-1.0,
                     use_bias=False, kernel_init=nn.initializers.he_uniform())(x[..., 0:8])


SyntaxError: ignored